In [8]:
import pandas as pd
import pyodbc
import pyarrow as pa
import pyarrow.parquet as pq
import os
import shutil
import glob
from datetime import date, datetime
from time import time
from azure.storage.blob import BlobServiceClient

ModuleNotFoundError: No module named 'azure'

In [2]:
ref_data = date.today()
ref_timestamp = time()

In [3]:
server = 'localhost'
database = 'db_datasus'
username = 'sa'
password = 'Ribeiro83'

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)

cursor = conn.cursor()

sql_query = pd.read_sql_query("""SELECT [date] 
                                     ,[state] 
                                     ,SUM([confirmed]) AS [confirmed] 
                                     ,SUM([deaths]) AS [deaths] 
                                     ,[estimated_population_2019] 
                                     ,[estimated_population] 
                                     ,[city_ibge_code] 
                                     ,[confirmed_per_100k_inhabitants] 
                                     ,[death_rate] 
                                FROM [db_datasus].[dbo].[COVID] 
                                WHERE city_ibge_code < 100 
                                GROUP BY  
                                      [date] 
                                      ,[state] 
                                      ,[estimated_population_2019] 
                                      ,[estimated_population] 
                                      ,[city_ibge_code] 
                                      ,[confirmed_per_100k_inhabitants] 
                                      ,[death_rate] 
                                ORDER BY [date], [state] ASC""", conn)

# sql_query

In [4]:
df = pd.DataFrame(sql_query)
# df

In [5]:
df.to_parquet('df.parquet.snappy',compression='snappy')
# pd.read_parquet('df.parquet.snappy')

In [6]:
table = pa.Table.from_pandas(df)

In [7]:
pq.write_to_dataset(table, root_path='{"CloudHub.Azure.Storage.BlobContainer":{"connection":{"type":"blobContainer","serviceEndpoint":"http://127.0.0.1:10000/devstoreaccount1","credentials":[{"type":"key","accountName":"devstoreaccount1","accountKey":"Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw=="}],"container":"dev-test"},"connectionString":"AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;DefaultEndpointsProtocol=http;BlobEndpoint=http://127.0.0.1:10000/devstoreaccount1;QueueEndpoint=http://127.0.0.1:10001/devstoreaccount1;TableEndpoint=http://127.0.0.1:10002/devstoreaccount1;","containerName":"dev-test","accountUri":"http://127.0.0.1:10000/devstoreaccount1","sasToken":"sv=2018-03-28&ss=b&srt=co&sp=rl&se=2021-04-29T17%3A37%3A08Z&sig=X2NMcubZr4Dlp1KYJQo%2BiUvIHYjK0mMXJNeoQh5Zg5o%3D"}}')

AssertionError: 

In [123]:
# date = str(ref_data)
# time = str(ref_timestamp)

# filename = date +'_'+ time+'_'

# pq.write_table(table, filename+'.parquet')

In [124]:
# source_dir = 'D:/OneDrive/Download'
# sources = glob.glob(os.path.join(source_dir,"*.parquet"))

# destination = 'C:/Users/tiago/Desktop/'+str(ref_data)

# for source in sources:
#     if os.path.isfile(source):
#         shutil.move(source,destination)